<a href="https://colab.research.google.com/github/kashishthakur26/MLprojects/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score , confusion_matrix

In [3]:
import pandas as pd

In [18]:
train_df = pd.read_csv("/content/drive/MyDrive/Colab-notebooks/train.csv",encoding='latin1')

In [19]:
train_df.head(100)

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my APL frie...
1,2,0,I missed the New Moon trail...
2,3,1,omg its already 7:30 :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,i think mi bf is cheating on me!!! ...
...,...,...,...
95,96,0,mi momacita won't let me go to my bf's bball...
96,97,0,Mom says I have to get a new phone IMMEDIATE...
97,98,0,My new car was stolen....by my mother who wa...
98,99,0,"no hang out with the girls 2day. 2moro, hope..."


In [13]:
# def preprocess_text(text):

#   text = text.lower()

#   text = text.translate(str.maketrans("","",string.punctuation))

#   tokens = word_tokenize(text)

#   stop_words = set(stopwords.words("english"))
#   filtered_tokens = [word for word in tokens if word not in stop_words]

#   preprocessed_text = " ".join(filtered_tokens)

#   return preprocesse text

In [17]:
# df["preprocessed_text"] = df["SentimentText"].apply(preprocess_text)
# #

LookupError: ignored

In [15]:
# pip install nltk

In [21]:
train_df.Sentiment.value_counts()

1    56457
0    43532
Name: Sentiment, dtype: int64

In [22]:
train_df_shuffled = train_df.sample(frac=1,random_state=42)

train_df_shuffled.head()

,ItemID,Sentiment,SentimentText
33965,33977,1,@Allieandra wheeee!
22853,22865,1,@a02toyota Thank you for the FF! Good to meet ya
19448,19460,0,@ electricbath Eewwww. Gross! So sorry hayward...
9732,9744,1,"#followfriday - I'm a little late, but here's ..."
7129,7132,1,"#icanhelp in shopping (deals), personal assist..."


In [23]:
import random
random_index = random.randint(0,len(train_df)-5)

for row in train_df_shuffled[["Sentiment","SentimentText"]][random_index:random_index+5].itertuples():
  _,Sentiment,SentimentText = row
  print(f"Target: {Sentiment}","(real diaster)" if Sentiment > 0 else "(not real disaster)")

  print(f"Text:\n{SentimentText}\n")
  print("____\n")


Target: 0 (not real disaster)
Text:
@buddalunch I hope i will be badass , But everyones starting to flake out  we'll see what happens . I'll update u later

____

Target: 0 (not real disaster)
Text:
@5windows still hate me  ???

____

Target: 1 (real diaster)
Text:
@_supernatural_ http://twitpic.com/68xmx - Because of my perking nipples !  Love the eyes 

____

Target: 0 (not real disaster)
Text:
@angeladegaitas or be a big fat bitch head  or at least come tomorrow!

____

Target: 0 (not real disaster)
Text:
@Cash07 Oh u didnt get me nuthing tooo   Forget yall then U and db0y8199.... lol

____



In [25]:
from sklearn.model_selection import train_test_split

train_sentences ,val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["SentimentText"].to_numpy(),
                                                                            train_df_shuffled["Sentiment"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [26]:
len(train_sentences) , len(val_sentences)

(89990, 9999)

## Converting the text into the numbers

In [27]:
import tensorflow as tf

from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

text_vectorizer = TextVectorization(max_tokens=15,
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None,
                                    output_mode="int",
                                    output_sequence_length=None,
                                    pad_to_max_tokens=True)

In [28]:
max_vocab_length = 10000
max_length = 15

In [29]:
text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [30]:
text_vectorizer.adapt(train_sentences)

In [31]:
train_sentences[:10]

array([" I'm all finished listening... I'm sorta considering giving it another play though. I might have missed a joke or two after all.",
       "&quot;I'm a celebrity get me out of here&quot; is dumb. hahaha &quot;spidey&quot; are full of themselves thankgosh they left, i'm rooting for Sanjaya ",
       '&quot;long live the King who died at the hands of his enemies that they might know what love looks like.&quot; ',
       "@avidbookreader I updated iTunes the other day (I dl from the Apple website and then install). Can't find cell cord so can't updte to 3.0 ",
       "@backstreetboys heey, sexy boys! when is your album goin to be readdy?  can't wait! ~KTBSPA",
       "@AGirlCalledKate Yep! I'm thinking Edward is going to be the winner tonight ",
       "@ButtercupD I will. Now I'm hungry for ice cream. ",
       "@Amanda_Holden Just had to say I LOVE your dresses on BGT, esp. tuesday's! Can't believe it's nearly over ",
       '      I must think about positive..',
       '@C_DIG w

In [32]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]

In [33]:
print(top_5_words)
print(bottom_5_words)

['', '[UNK]', 'i', 'the', 'to']
['amoyal', 'amon91', 'ammiisaurusrexx', 'amethystgurl07', 'ameliagrace']


##Creating an embeddings using an embedding layer

In [34]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             input_length = max_length)
embedding

In [36]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\\n\n embedded version: ")

sample_embedd = embedding(text_vectorizer([random_sentence]))
sample_embedd

Original text:
@bullyinguk I can view the homepage but my inbox doesn't seem to be working - I was in the middle of sending a message when it happened \n
 embedded version: 


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.04553595,  0.03934593,  0.04420074, ...,  0.00753683,
         -0.04205842, -0.01495727],
        [ 0.04506769,  0.03589118,  0.04210155, ...,  0.00190227,
          0.04810372,  0.04067004],
        [ 0.01177637,  0.00209637, -0.002769  , ..., -0.04060886,
          0.04108561,  0.02792463],
        ...,
        [-0.03464408, -0.0100451 , -0.03263484, ..., -0.02700492,
         -0.03225817, -0.03852032],
        [-0.04346178, -0.04198145, -0.00309708, ..., -0.01365572,
          0.0034384 , -0.04475298],
        [ 0.04506769,  0.03589118,  0.04210155, ...,  0.00190227,
          0.04810372,  0.04067004]]], dtype=float32)>

## Naive Bayes

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ("tfidf",TfidfVectorizer()),
    ("clf",MultinomialNB())
])

model_0.fit(train_sentences , train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [38]:
score = model_0.score(val_sentences , val_labels)

In [39]:
score

0.7477747774777478